In [1]:
!which python; python -V;

# This makes the diagrams to more reliably appear in Jupyter environment
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# This will cause the ephemerides to be imported from JPL horizons system
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

/home/thomson/devel/perylune/venv/bin/python
Python 3.8.5


<ScienceState solar_system_ephemeris: 'jpl'>

In [2]:
from poliastro.bodies import Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, Sun, Pluto
from poliastro.twobody import Orbit
from poliastro.constants import GM_earth, GM_sun
from poliastro.frames import Planes
from poliastro.maneuver import Maneuver

from perylune.orbit_tools import *
from perylune.interplanetary import *
from perylune.constants import *
from perylune.horizons import *

from astropy import units as u
from astropy import time
import numpy as np

In [3]:
from poliastro.ephem import Ephem
from poliastro.util import time_range
import plotly.graph_objs as go

def heliocentric_velocity(orbit):

    if orbit.attractor != Sun:
        gm = G * orbit.attractor.mass
    else:
        gm = GM_sun

    # We could use the current distance (orbit.r), periapsis or apoapsis (orbit.r_a or orbit.r_p),
    # but we'll simply go with a = (r_a + r_p) / 2
    a = orbit.a.to(u.m)

    v = np.sqrt(gm*(1.0/a))

    return v

def transfer_delta_v(body1, body2, attractor):
    """Returns transfer parameters for body1 (e.g. Earth) to body2 (e.g. Mars).
       Optionally, the main attractor can be specified. If omitted, Sun is assumed."""

    # How to obtain the 
    method = "horizons_orbit" # allowed values are ephem, horizons_orbit

    if attractor is None:
        attractor = Sun

    from astropy import time
    from poliastro.ephem import Ephem
    from poliastro.util import time_range

    # Let's assume the calculations are done for 2020.
    date_start = time.Time("2020-01-01 00:00", scale="utc").tdb
    date_end =   time.Time("2021-12-31 23:59", scale="utc").tdb

    name1, id_type1 = name_to_horizons_id(body1)
    name2, id_type2 = name_to_horizons_id(body2)

    if method == "ephem":
        # Get the ephemerides first and then contruct orbit based on them. This is the recommended
        # way. See warning in Orbit.from_horizons about deprecation.
        ephem1 = Ephem.from_horizons(name=name1, epochs=time_range(date_start, end=date_end), plane=Planes.EARTH_ECLIPTIC, id_type=id_type1)
        ephem2 = Ephem.from_horizons(name=name2, epochs=time_range(date_start, end=date_end), plane=Planes.EARTH_ECLIPTIC, id_type=id_type2)

        # Solve for departure and target orbits
        orb1 = Orbit.from_ephem(Sun, ephem1, date_start + 180 * u.day)
        orb2 = Orbit.from_ephem(Sun, ephem2, date_end)
    elif method == "horizons_orbit":
        # This is the old way. Sadly, it produces way better values.
        orb1 = Orbit.from_horizons(name=name1, attractor=attractor, plane=Planes.EARTH_ECLIPTIC, id_type=id_type1)
        orb2 = Orbit.from_horizons(name=name2, attractor=attractor, plane=Planes.EARTH_ECLIPTIC, id_type=id_type2)
    else:
        raise "Invalid method set."

    print("=== Departure ===")
    print_orb(orb1)

    print("=== Arrival ===")
    print_orb(orb2)

    # This returns complete nonsense.
    # man_lambert = Maneuver.lambert(orb1, orb2)
    hoh = Maneuver.hohmann(orb1, orb2.a)

    orb_trans, orb_target = orb1.apply_maneuver(hoh, intermediate=True)
    print(hoh)
    print("Total transfer time=%s" % hoh.get_total_time().to(u.day))
    print(np.linalg.norm(orb_trans.v))

    # The escape_delta_v returns a tuple of escape velocity at current, periapsis, apoapsis.
    helio1 = heliocentric_velocity(orb1)
    helio2 = heliocentric_velocity(orb2)

    vesc1 = escape_delta_v(orb1, False)[1]
    vesc2 = escape_delta_v(orb2, False)[1]

    return helio1, vesc1, helio2, vesc2

In [4]:
# Calculate delta-v necessary to reach escape velocity for circular orbit. The values returned are current position, periapsis, apoapis.
# No surprises here - it's perfectly circular, so it's always the same.
#escape_delta_v(orb1, False)
transfer_delta_v("earth", "mars", None)

=== Departure ===
1 x 1 AU x 0.0 deg (HeliocentricEclipticIAU76) orbit around Sun (☉) at epoch 2020-11-06 17:08:28.420765 (TDB)
a(𝑎)=0.9997AU, b=0.9996AU, e=0.02, i=0.00deg raan(Ω)=193.87deg argp(𝜔)=269.35deg nu(𝜈)=-58.76deg
period=31543579.43s perapis=0.9834AU(0.98AU) apoapsis=1.0160AU(1.01AU)
=== Arrival ===
1 x 2 AU x 1.8 deg (HeliocentricEclipticIAU76) orbit around Sun (☉) at epoch 2020-11-06 17:08:29.426534 (TDB)
a(𝑎)=1.5236AU, b=1.5170AU, e=0.09, i=1.85deg raan(Ω)=49.49deg argp(𝜔)=286.62deg nu(𝜈)=58.76deg
period=59350948.25s perapis=1.3814AU(1.38AU) apoapsis=1.6658AU(1.66AU)
Number of impulses: 2, Total cost: 5.591875 km / s
Total transfer time=314.29733692671033 d
33.11262802904298 km / s


(<Quantity 29789.29179516 m / s>,
 <Quantity 12191.98390788 m / s>,
 <Quantity 24129.84964154 m / s>,
 <Quantity 9335.91417245 m / s>)